In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("card_cust.csv")
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


### 전처리

In [4]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [5]:
df.isna().sum()

CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

In [6]:
df_base = df.reset_index(drop = True)
len(df_base)

1000

### Q1.

In [7]:
df_q1 = df_base[["TENURE", "BALANCE", "CREDIT_LIMIT"]].copy()
df_q1.head(2)

,TENURE,BALANCE,CREDIT_LIMIT
0,12.0,40.900749,1000.0
1,12.0,3202.467416,7000.0


In [9]:
df_q1_corr = df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()
df_q1_corr.head()

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696

In [10]:
df_q1_corr.reset_index()

,TENURE,level_1,BALANCE,CREDIT_LIMIT
0,6.0,BALANCE,1.000000,0.868056
1,6.0,CREDIT_LIMIT,0.868056,1.000000
2,7.0,BALANCE,1.000000,0.948405
3,7.0,CREDIT_LIMIT,0.948405,1.000000
4,8.0,BALANCE,1.000000,0.820696
5,8.0,CREDIT_LIMIT,0.820696,1.000000
6,9.0,BALANCE,1.000000,0.085474
7,9.0,CREDIT_LIMIT,0.085474,1.000000
8,10.0,BALANCE,1.000000,0.291482
9,10.0,CREDIT_LIMIT,0.291482,1.000000


In [13]:
df_q1_corr.xs("BALANCE", level = 1)["CREDIT_LIMIT"].round(2).max()

0.95

### Q2.

In [ ]:
df_q2 = df_base.drop(columns = "CUST_ID")
df_q2.head(1)

In [16]:
arr_q2_nor = StandardScaler().fit_transform(df_q2)
df_q2_nor = pd.DataFrame(arr_q2_nor, columns = df_q2.columns)
df_q2_nor.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


In [25]:
ls_k = [2, 3, 4, 5]
k = ls_k[0]

model_kmeans = KMeans(n_clusters = k, random_state = 1234)
model_kmeans.fit(df_q2_nor)
val_sil = silhouette_score(df_q2_nor, labels = model_kmeans.labels_)
val_sil

0.3075281530456079

In [ ]:
ls_k = [2, 3, 4, 5]

ls_sil = []
for k in ls_k:
    model_kmeans = KMeans(n_clusters = k, random_state = 1234)
    model_kmeans.fit(df_q2_nor)
    val_sil = silhouette_score(df_q2_nor, labels = model_kmeans.labels_)
    ls_sil = ls_sil + [val_sil]

In [32]:
ser_sil = pd.Series(ls_sil, index = ls_k)
# ser_sil.plot.bar()

In [ ]:
best_k = ser_sil.idxmax()
best_k

In [ ]:
k

In [ ]:
model_kmeans_best = KMeans(n_clusters = best_k, random_state = 1234)
model_kmeans_best.fit(df_q2_nor)

In [36]:
df_q2["cluster"] = model_kmeans_best.labels_

In [39]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2).max()

3946.19

### Q3.

In [40]:
7 % 5

2

In [41]:
df_train = df_base.loc[df_base["CUST_ID"] % 4 != 0, ]
df_test  = df_base.loc[df_base["CUST_ID"] % 4 == 0, ]
len(df_train), len(df_test)

(752, 248)

In [43]:
model_dt = DecisionTreeRegressor(random_state = 1234)
model_dt.fit(X = df_train.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]),
             y = df_train["ONEOFF_PURCHASES"])
pred = model_dt.predict(df_test.drop(columns = ["CUST_ID", "ONEOFF_PURCHASES"]))
pred[:4]

array([1500.,    0., 1490.,    0.])

In [44]:
y_t = df_test["ONEOFF_PURCHASES"]
y_p = pred

In [51]:
# y_t - y_p # Error
# (y_t - y_p) ** 2 # Squared Error
# ((y_t - y_p) ** 2).mean() # Mean Squared Error
((y_t - y_p) ** 2).mean() ** 0.5 # Root Mean Squared Error

1039.193967231063

In [53]:
round(((y_t - y_p) ** 2).mean() ** 0.5, 1)

1039.2

In [46]:
from sklearn.metrics import mean_squared_error as MSE

In [54]:
round(MSE(y_true = y_t, y_pred = y_p) ** 0.5, 1)

1039.2